# Projekat: Automatska klasifikacija proizvoda po kategorijama

## Uvod
Ovaj projekat ima za cilj razvoj ML modela koji automatski predviđa kategoriju proizvoda na osnovu njegovog naziva.  
Sveska je organizovana tako da bilo koji član tima može pratiti tok projekta i nastaviti rad, bez dodatnih objašnjenja.

---

## Korak 1 – Kreiranje GitHub repozitorijuma
- Napravljen javni repozitorijum na GitHub-u sa nazivom `product-category-classifier`
- Inicijalni README.md fajl sa osnovnim opisom projekta

## Korak 2 – Kloniranje repozitorijuma lokalno
- Repozitorijum je kloniran na lokalni računar
- Obezbeđena je sinhronizacija između lokalnog foldera i GitHub repozitorijuma

## Korak 3 – Kreiranje Jupyter/Colab radne sveske
- Napravljen folder `notebooks` unutar repozitorijuma
- Kreirana radna sveska `product_category_analysis.ipynb` u Colab-u
- Ova sveska sadrži kompletnu analizu, pripremu podataka i razvoj ML modela


In [ ]:
# ===============================
# KORAK 4: UČITAVANJE I ISTRAŽIVANJE PODATAKA
# ===============================

# Uvoz potrebnih biblioteka
import pandas as pd
from google.colab import files

# ---------------------------------------------------
# 4.1 Upload CSV fajla
# ---------------------------------------------------
# Funkcija files.upload() omogućava interaktivno biranje fajla sa lokalnog računara.
# Ovde ćemo izabrati 'products.csv'.
uploaded = files.upload()

# ---------------------------------------------------
# 4.2 Učitavanje CSV fajla u Pandas DataFrame
# ---------------------------------------------------
# Pandas DataFrame je struktura podataka pogodna za analizu i obradu tabularnih podataka.
df = pd.read_csv("products.csv")

# ---------------------------------------------------
# 4.2b Očistiti imena kolona
# ---------------------------------------------------
# Uklanjamo vodeće i prateće razmake u imenima kolona kako bismo izbegli KeyError u budućim koracima
df.columns = df.columns.str.strip()

# Prikaz prvih 5 redova da vizuelno proverimo strukturu podataka
df.head()

# ---------------------------------------------------
# 4.3 Pregled osnovnih informacija o dataset-u
# ---------------------------------------------------
# df.info() prikazuje:
# - broj redova i kolona
# - tipove podataka po kolonama
# - broj nenultih vrednosti po koloni
df.info()

# df.describe() daje statistički pregled numeričkih kolona
# (proseci, min, max, standardna devijacija, percentili)
df.describe()

# ---------------------------------------------------
# 4.4 Pregled broja proizvoda po kategoriji
# ---------------------------------------------------
# Ovo pomaže timu da razume raspodelu ciljnih klasa i potencijalnu neravnotežu
df['Category Label'].value_counts()


Saving products.csv to products (2).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35311 entries, 0 to 35310
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product ID       35311 non-null  int64  
 1   Product Title    35139 non-null  object 
 2   Merchant ID      35311 non-null  int64  
 3   Category Label   35267 non-null  object 
 4   _Product Code    35216 non-null  object 
 5   Number_of_Views  35297 non-null  float64
 6   Merchant Rating  35141 non-null  float64
 7   Listing Date     35252 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.2+ MB


,count
Category Label,
Fridge Freezers,5495
Washing Machines,4036
Mobile Phones,4020
CPUs,3771
TVs,3564
Fridges,3457
Dishwashers,3418
Digital Cameras,2696
Microwaves,2338


In [ ]:
# ===============================
# KORAK 5: PRIPREMA I ČIŠĆENJE PODATAKA
# ===============================

# ---------------------------------------------------
# 5.1 Provera praznih vrednosti po kolonama
# ---------------------------------------------------
# df.isnull().sum() vraća broj praznih vrednosti za svaku kolonu
print("Prazne vrednosti po kolonama:\n", df.isnull().sum())

# ---------------------------------------------------
# 5.2 Selektovanje kolona koje će se koristiti za model
# ---------------------------------------------------
# Za predikciju kategorije koristićemo samo naziv proizvoda
# Ostale kolone možemo koristiti kasnije za feature engineering
df_model = df[['Product Title', 'Category Label']].copy()

# ---------------------------------------------------
# 5.3 Uklanjanje redova sa praznim nazivima proizvoda ili kategorijama
# ---------------------------------------------------
# Model ne može učiti iz praznih vrednosti, pa ih uklanjamo
df_model = df_model.dropna(subset=['Product Title', 'Category Label'])

# ---------------------------------------------------
# 5.4 Standardizacija teksta (svi karakteri mala slova)
# ---------------------------------------------------
# Ovo pomaže modelu da prepozna iste reči bez obzira na velika/mala slova
df_model['Product Title'] = df_model['Product Title'].str.lower()

# ---------------------------------------------------
# 5.5 Resetovanje indeksa nakon uklanjanja redova
# ---------------------------------------------------
df_model = df_model.reset_index(drop=True)

# Provera finalnog dataset-a
print("Broj redova nakon čišćenja:", len(df_model))
df_model.head()


Prazne vrednosti po kolonama:
 product ID           0
Product Title      172
Merchant ID          0
Category Label      44
_Product Code       95
Number_of_Views     14
Merchant Rating    170
Listing Date        59
dtype: int64
Broj redova nakon čišćenja: 35096


,Product Title,Category Label
0,apple iphone 8 plus 64gb silver,Mobile Phones
1,apple iphone 8 plus 64 gb spacegrau,Mobile Phones
2,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Mobile Phones
3,apple iphone 8 plus 64gb space grey,Mobile Phones
4,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Mobile Phones


In [ ]:
# ===============================
# KORAK 6: FEATURE ENGINEERING
# ===============================

# Kreiranje novih kolona koje mogu pomoći modelu:

# ---------------------------------------------------
# 6.1 Broj reči u nazivu proizvoda
# ---------------------------------------------------
# Ovo može pomoći da model razlikuje kratke i duže naslove proizvoda
df_model['Title_Word_Count'] = df_model['Product Title'].str.split().str.len()

# ---------------------------------------------------
# 6.2 Dužina naziva proizvoda (broj karaktera)
# ---------------------------------------------------
# Pomaže modelu da prepozna duže nazive koji često sadrže više detalja
df_model['Title_Char_Count'] = df_model['Product Title'].str.len()

# ---------------------------------------------------
# 6.3 Prisustvo brojeva u nazivu proizvoda
# ---------------------------------------------------
# Može pomoći modelu da prepozna modele, kapacitete, GB, MP itd.
df_model['Contains_Number'] = df_model['Product Title'].str.contains(r'\d').astype(int)

# ---------------------------------------------------
# 6.4 Broj specijalnih znakova
# ---------------------------------------------------
# Specijalni znakovi kao '/' ili '-' često nose dodatne informacije
df_model['Special_Char_Count'] = df_model['Product Title'].str.count(r'[^a-zA-Z0-9 ]')

# ---------------------------------------------------
# 6.5 Prikaz prvih redova sa novim karakteristikama
# ---------------------------------------------------
df_model.head()


,Product Title,Category Label,Title_Word_Count,Title_Char_Count,Contains_Number,Special_Char_Count
0,apple iphone 8 plus 64gb silver,Mobile Phones,6,31,1,0
1,apple iphone 8 plus 64 gb spacegrau,Mobile Phones,7,35,1,0
2,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Mobile Phones,13,70,1,2
3,apple iphone 8 plus 64gb space grey,Mobile Phones,7,35,1,0
4,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Mobile Phones,11,54,1,1


In [ ]:
# ===============================
# 7. Uporedi ML modele i sačuvaj najbolji
# ===============================

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# --- 7.1 Podela podataka na X i y ---
X = df_model[['Product Title', 'Title_Word_Count', 'Title_Char_Count', 'Contains_Number', 'Special_Char_Count']]
y = df_model['Category Label']

# --- 7.2 Podela na trening i test set ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 7.3 Preprocesiranje ---
# TF-IDF za tekst, StandardScaler za numeričke kolone
preprocessor = ColumnTransformer(transformers=[
    ('tfidf', TfidfVectorizer(), 'Product Title'),
    ('num', StandardScaler(), ['Title_Word_Count', 'Title_Char_Count', 'Contains_Number', 'Special_Char_Count'])
])

# --- 7.4 Definisanje modela ---
models = {
    "Logistic Regression": Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    "Random Forest": Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
    ]),
    "Gradient Boosting": Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(n_estimators=200, random_state=42))
    ])
}

# --- 7.5 Treniranje i evaluacija ---
best_accuracy = 0
best_model = None

for name, model in models.items():
    print(f"=== {name} ===")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    report = classification_report(y_test, y_pred, output_dict=True)
    print(f"Accuracy: {report['accuracy']:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("----------------------------------------")

    # --- Čuvanje najboljeg modela ---
    if report['accuracy'] > best_accuracy:
        best_accuracy = report['accuracy']
        best_model = model

print(f"Najbolji model za čuvanje: {best_model.named_steps['classifier'].__class__.__name__} sa tačnošću {best_accuracy:.4f}")

# --- 7.6 Sačuvaj najbolji model ---
joblib.dump(best_model, "saved_models/best_product_category_model.pkl")
print("Najbolji model je uspešno sačuvan u: saved_models/best_product_category_model.pkl")


=== Logistic Regression ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

Accuracy: 0.9543
Classification Report:
                  precision    recall  f1-score   support

             CPU       0.00      0.00      0.00        17
            CPUs       0.98      1.00      0.99       749
 Digital Cameras       1.00      0.99      0.99       538
     Dishwashers       0.93      0.96      0.94       681
        Freezers       1.00      0.93      0.96       440
 Fridge Freezers       0.95      0.94      0.94      1094
         Fridges       0.86      0.91      0.88       687
      Microwaves       0.99      0.95      0.97       466
    Mobile Phone       0.00      0.00      0.00        11
   Mobile Phones       0.96      0.99      0.98       801
             TVs       0.97      0.99      0.98       708
Washing Machines       0.95      0.95      0.95       803
          fridge       0.00      0.00      0.00        25

        accuracy                           0.95      7020
       macro avg       0.74      0.74      0.74      7020
    weighted avg       0.95   

In [ ]:
# ===============================
# KORAK 8: ČUVANJE NAJBOLJEG MODELA
# ===============================

import joblib
import os

# Folder gde ćemo sačuvati model
model_folder = "saved_models"
os.makedirs(model_folder, exist_ok=True)

# Putanja do fajla
model_path = os.path.join(model_folder, "best_product_category_model.pkl")

# Sačuvaj model iz koraka 7 (best_model)
joblib.dump(best_model, model_path)

print(f"Najbolji model ({best_model.named_steps['classifier'].__class__.__name__}) je uspešno sačuvan u: {model_path}")


Najbolji model (RandomForestClassifier) je uspešno sačuvan u: saved_models/best_product_category_model.pkl


In [ ]:
# ===============================
# KORAK 9: Test primeri proizvoda sa ✅/❌
# ===============================

import pandas as pd
import joblib

# Učitavanje najboljeg sačuvanog modela
best_model = joblib.load("saved_models/best_product_category_model.pkl")

# Lista test proizvoda i očekivanih kategorija
test_products = [
    {"Product Title": "iphone 7 32gb gold", "Expected Category": "Mobile Phones"},
    {"Product Title": "olympus e m10 mark iii geh use silber", "Expected Category": "Digital Cameras"},
    {"Product Title": "kenwood k20mss15 solo", "Expected Category": "Microwaves"},
    {"Product Title": "bosch wap28390gb 8kg 1400 spin", "Expected Category": "Washing Machines"},
    {"Product Title": "bosch serie 4 kgv39vl31g", "Expected Category": "Fridge Freezers"},
    {"Product Title": "smeg sbs8004po", "Expected Category": "Fridge Freezers"}
]

# Kreiranje DataFrame-a
df_test = pd.DataFrame(test_products)

# Feature engineering za test proizvode
df_test['Title_Word_Count'] = df_test['Product Title'].str.split().str.len()
df_test['Title_Char_Count'] = df_test['Product Title'].str.len()
df_test['Contains_Number'] = df_test['Product Title'].str.contains(r'\d').astype(int)
df_test['Special_Char_Count'] = df_test['Product Title'].str.count(r'[^a-zA-Z0-9 ]')

# Predikcija kategorija
df_test['Predicted Category'] = best_model.predict(df_test[['Product Title', 'Title_Word_Count',
                                                           'Title_Char_Count', 'Contains_Number',
                                                           'Special_Char_Count']])

# Provera tačnosti
df_test['Correct'] = df_test['Predicted Category'] == df_test['Expected Category']
df_test['Correct'] = df_test['Correct'].apply(lambda x: "✅" if x else "❌")

# Prikaz tabele sa rezultatima
df_test


,Product Title,Expected Category,Title_Word_Count,Title_Char_Count,Contains_Number,Special_Char_Count,Predicted Category,Correct
0,iphone 7 32gb gold,Mobile Phones,4,18,1,0,Mobile Phones,✅
1,olympus e m10 mark iii geh use silber,Digital Cameras,8,37,1,0,Digital Cameras,✅
2,kenwood k20mss15 solo,Microwaves,3,21,1,0,Microwaves,✅
3,bosch wap28390gb 8kg 1400 spin,Washing Machines,5,30,1,0,Washing Machines,✅
4,bosch serie 4 kgv39vl31g,Fridge Freezers,4,24,1,0,Dishwashers,❌
5,smeg sbs8004po,Fridge Freezers,2,14,1,0,Dishwashers,❌


In [ ]:
# -------------------------------
# 10.1: train_model.py
# -------------------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib
import os

# Učitavanje podataka
df = pd.read_csv("products.csv")
df.columns = df.columns.str.strip()

# Priprema i čišćenje podataka
df_model = df[['Product Title', 'Category Label']].dropna()
df_model['Product Title'] = df_model['Product Title'].str.lower()
df_model = df_model.reset_index(drop=True)

# Feature engineering
df_model['Title_Word_Count'] = df_model['Product Title'].str.split().str.len()
df_model['Title_Char_Count'] = df_model['Product Title'].str.len()
df_model['Contains_Number'] = df_model['Product Title'].str.contains(r'\d').astype(int)
df_model['Special_Char_Count'] = df_model['Product Title'].str.count(r'[^a-zA-Z0-9 ]')

# Podela na X i y
X = df_model[['Product Title', 'Title_Word_Count', 'Title_Char_Count', 'Contains_Number', 'Special_Char_Count']]
y = df_model['Category Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Preprocesor
preprocessor = ColumnTransformer(transformers=[
    ('tfidf', TfidfVectorizer(), 'Product Title'),
    ('num', StandardScaler(), ['Title_Word_Count', 'Title_Char_Count', 'Contains_Number', 'Special_Char_Count'])
])

# Definisanje modela
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42))
])

# Treniranje
pipeline.fit(X_train, y_train)

# Čuvanje modela
model_folder = "saved_models"
os.makedirs(model_folder, exist_ok=True)
model_path = os.path.join(model_folder, "best_product_category_model.pkl")
joblib.dump(pipeline, model_path)
print(f"Najbolji model je uspešno sačuvan u: {model_path}")


Najbolji model je uspešno sačuvan u: saved_models/best_product_category_model.pkl


In [ ]:
# -------------------------------
# 10.2: predict_category.py
# -------------------------------
import joblib
import pandas as pd

# Učitavanje sačuvanog modela
model_path = "saved_models/best_product_category_model.pkl"
pipeline = joblib.load(model_path)
print("Model uspešno učitan!")

# Funkcija za predikciju kategorije proizvoda
def predict_category(title):
    df_input = pd.DataFrame({
        'Product Title': [title],
        'Title_Word_Count': [len(title.split())],
        'Title_Char_Count': [len(title)],
        'Contains_Number': [int(any(c.isdigit() for c in title))],
        'Special_Char_Count': [sum(not c.isalnum() and not c.isspace() for c in title)]
    })
    category = pipeline.predict(df_input)[0]
    return category

# Interaktivno testiranje
while True:
    user_input = input("Unesite naziv proizvoda (ili 'exit' za kraj): ").strip()
    if user_input.lower() == 'exit':
        break
    predicted_category = predict_category(user_input.lower())
    print(f"Predviđena kategorija: {predicted_category}")


Model uspešno učitan!
Unesite naziv proizvoda (ili 'exit' za kraj): kenwood k20mss15 solo
Predviđena kategorija: Microwaves
Unesite naziv proizvoda (ili 'exit' za kraj): bosch serie 4 kgv39vl31g
Predviđena kategorija: Dishwashers
Unesite naziv proizvoda (ili 'exit' za kraj): iphone 13 pro max 256gb
Predviđena kategorija: Mobile Phones
Unesite naziv proizvoda (ili 'exit' za kraj): exit
